# [12 Python Decorators to Take Your Code to the Next Level](https://towardsdatascience.com/12-python-decorators-to-take-your-code-to-the-next-level-a910a1ab3e99)

## 1 — @logger (to get started)✏️

Если вы впервые знакомы с декораторами, то можете считать их функциями, которые принимают функции на вход и расширяют их функциональные возможности без изменения их основного назначения.

Давайте начнем с простого декоратора, который расширяет функцию, регистрируя время начала и окончания ее выполнения.

Результат работы декорируемой функции будет выглядеть следующим образом:

# ----- some_function: start -----
# some_function executing
# ----- some_function: end -----
Чтобы написать этот дешифратор, сначала нужно выбрать подходящее имя: назовем его logger.

Логгер - это функция, которая принимает функцию на вход и возвращает функцию на выход. Выходная функция обычно является расширенной версией входной. В нашем случае мы хотим, чтобы выходная функция окружала вызов входной функции утверждениями start и end.

Поскольку мы не знаем, какие аргументы использует входная функция, мы можем передать их из функции-обертки с помощью выражений *args и **kwargs. Эти выражения позволяют передавать произвольное количество позиционных и ключевых аргументов.

Вот простая реализация декоратора регистратора:

In [9]:

def logger(function):
    def wrapper(*args, **kwargs):
        print(f"----- {function.__name__}: start -----")
        output = function(*args, **kwargs)
        print(f"----- {function.__name__}: end -----")
        return output
    return wrapper

@logger
def some_function(text):
    print(text)

some_function("first test")
# ----- some_function: start -----
# first test
# ----- some_function: end -----
some_function("second test")
# ----- some_function: start -----
# second test
# ----- some_function: end -----


----- some_function: start -----
first test
----- some_function: end -----
----- some_function: start -----
second test
----- some_function: end -----


## 2 — @wraps 🎁
Этот декоратор обновляет функцию-обертку, чтобы она выглядела как оригинальная функция и наследовала ее имя и свойства.

Чтобы понять, что делает @wraps и почему вы должны его использовать, давайте возьмем предыдущий декоратор и применим его к простой функции, которая складывает два числа.

In [10]:
def logger(function):
    def wrapper(*args, **kwargs):
        """wrapper documentation"""
        print(f"----- {function.__name__}: start -----")
        output = function(*args, **kwargs)
        print(f"----- {function.__name__}: end -----")
        return output
    return wrapper

@logger
def add_two_numbers(a, b):
    """this function adds two numbers"""
    return a + b

In [5]:
print(add_two_numbers.__name__)

print(add_two_numbers.__doc__)

wrapper
wrapper documentation


Если мы проверим имя и документацию декорированной функции add_two_numbers, вызвав атрибуты `__name__` и `__doc__`, мы получим ... неестественные (и все же ожидаемые) результаты:

Вместо этого мы получаем имя обертки и документацию ⚠️.

Это нежелательный результат. Мы хотим сохранить имя и документацию исходной функции. Вот тут-то и пригодится декоратор @wraps.

Все, что вам нужно сделать, это украсить функцию-обертку.

In [11]:
from functools import wraps

def logger(function):
    @wraps(function)
    def wrapper(*args, **kwargs):
        """wrapper documentation"""
        print(f"----- {function.__name__}: start -----")
        output = function(*args, **kwargs)
        print(f"----- {function.__name__}: end -----")
        return output
    return wrapper

@logger
def add_two_numbers(a, b):
    """this function adds two numbers"""
    return a + b

In [7]:
print(add_two_numbers.__name__)

print(add_two_numbers.__doc__)

add_two_numbers
this function adds two numbers


## 3 — @lru_cache 💨

Это встроенный декоратор, который можно импортировать из functools.

Он кэширует возвращаемые значения функции, используя алгоритм наименее часто используемых значений (LRU) для отбрасывания наименее используемых значений, когда кэш заполняется.

Обычно я использую этот декоратор для длительно выполняющихся задач, которые не меняют результат при одинаковых входных данных, например, запрос к базе данных, запрос статической удаленной веб-страницы или тяжелая обработка.

В следующем примере я использую lru_cache для декорирования функции, которая имитирует некоторую обработку. Затем я применяю функцию на одном и том же входе несколько раз подряд.

In [12]:
import random
import time
from functools import lru_cache


@lru_cache(maxsize=None)
def heavy_processing(n):
    sleep_time = n + random.random()
    time.sleep(sleep_time)

# first time
%%time
heavy_processing(0)
# CPU times: user 363 µs, sys: 727 µs, total: 1.09 ms
# Wall time: 694 ms

# second time
%%time
heavy_processing(0)
# CPU times: user 4 µs, sys: 0 ns, total: 4 µs
# Wall time: 8.11 µs

# third time
%%time
heavy_processing(0)
# CPU times: user 5 µs, sys: 1 µs, total: 6 µs
# Wall time: 7.15 µs

UsageError: Line magic function `%%time` not found.


In [5]:
%%time
heavy_processing(1)

CPU times: total: 0 ns
Wall time: 1.43 s


In [6]:
%%time
heavy_processing(1)

CPU times: total: 0 ns
Wall time: 0 ns


## 4 — @repeat 🔁
Этот декоратор заставляет функцию вызываться несколько раз подряд.

Это может быть полезно для отладки, стресс-тестов или автоматизации повторения нескольких задач.

В отличие от предыдущих декораторов, этот ожидает входной параметр.

In [7]:
def repeat(number_of_times):
    def decorate(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(number_of_times):
                func(*args, **kwargs)
        return wrapper
    return decorate

В следующем примере определяется декоратор repeat, который принимает в качестве аргумента число раз. Затем декоратор определяет функцию wrapper, которая оборачивается вокруг декорируемой функции. Функция-обертка вызывает декорируемую функцию количество раз, равное указанному числу.

In [14]:
@repeat(5)
def dummy():
    print("hello")

dummy()

hello
hello
hello
hello
hello


## 5 — @timeit ⏲️
Этот декоратор измеряет время выполнения функции и выводит результат: это служит для отладки или мониторинга.

В следующем фрагменте декоратор timeit измеряет время выполнения функции process_data и выводит прошедшее время в секундах.

In [15]:
import time
from functools import wraps

def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'{func.__name__} took {end - start:.6f} seconds to complete')
        return result
    return wrapper

@timeit
def process_data():
    time.sleep(1)

process_data()
# process_data took 1.000012 seconds to complete

process_data took 1.000671 seconds to complete


## 6 — @retry 🔁
Этот декоратор заставляет функцию повторять попытку несколько раз, когда она сталкивается с исключением.

Он принимает три аргумента: `количество повторных попыток, исключение, которое нужно поймать и повторить, и время сна между повторными попытками`.

Это работает следующим образом:

Функция-обертка запускает цикл for с количеством итераций num_retries.
На каждой итерации она вызывает входную функцию в блоке try/except. При успешном вызове она прерывает цикл и возвращает результат. В противном случае он спит в течение sleep_time секунд и переходит к следующей итерации.
Если после завершения цикла for вызов функции не удается, функция-обертка вызывает исключение.

In [31]:
import random
import time
from functools import wraps

def retry(num_retries, exception_to_check, sleep_time=0):
    """
    Decorator that retries the execution of a function if it raises a specific exception.
    """
    def decorate(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for i in range(1, num_retries+1):
                try:
                    return func(*args, **kwargs)
                except exception_to_check as e:
                    print(f"{func.__name__} raised {e.__class__.__name__}. Retrying...")
                    if i < num_retries:
                        time.sleep(sleep_time)
            # Raise the exception if the function was not successful after the specified number of retries
            raise e
        return wrapper
    return decorate

@retry(num_retries=2, exception_to_check=ValueError, sleep_time=5)
def random_value():
    value = random.randint(1, 5)
    if value == 3:
        raise ValueError("Value cannot be 3")
    return value

random_value()
# random_value raised ValueError. Retrying...
# 1

random_value()
# 5

5

## 7 — @countcall 🔢
Этот декоратор подсчитывает, сколько раз была вызвана функция.

Это число хранится в атрибуте обертки count .

In [34]:
from functools import wraps

def countcall(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        result = func(*args, **kwargs)
        print(f'{func.__name__} has been called {wrapper.count} times')
        return result
    wrapper.count = 0
    return wrapper

@countcall
def process_data():
    pass

process_data()
# process_data has been called 1 times
process_data()
# process_data has been called 2 times
process_data()
# process_data has been called 3 times

process_data has been called 1 times
process_data has been called 2 times
process_data has been called 3 times
